In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mymodel40/keras/default/1/my_model_epoch40.keras
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/metadata.csv
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/vest_dress_1393.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/long_sleeve_top_1802.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/short_sleeve_top_3193.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/shorts_2975.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/vest_dress_847.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/shorts_2347.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/long_sleeve_top_1493.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/long_sleeve_dress_1088.jpg
/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images/skirt_1633.jpg
/kaggle/input/garmentiq-classifi

In [2]:
#Importing Libraries
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers

2025-05-17 03:49:32.360101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747453772.613001      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747453772.682616      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Data Preprocessing and Loading**

In [3]:
#Loading the CSV and mapping images to labels
df = pd.read_csv('/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/metadata.csv')  # CSV with filename and label
df['filepath'] = df['filename'].apply(lambda x: os.path.join('/kaggle/input/garmentiq-classification-set-nordstrom-and-myntra/images', x))

In [ ]:
#Checking for missing files if any
missing_files = df[~df['filepath'].apply(os.path.exists)]
print(f"Missing files:\n{missing_files}")

In [4]:
#Generating train and test dataset
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['garment'], random_state=42)
print(f"Training dataset size: {train_df.shape}")
print(f"Testing dataset size: {test_df.shape}")

Training dataset size: (18612, 5)
Testing dataset size: (4654, 5)


In [5]:
#Organizing images into subfolders of garment types
import shutil

def copy_images(df, split):
    for _, row in df.iterrows():
        label_folder = os.path.join('data', split, row['garment'])
        os.makedirs(label_folder, exist_ok=True)
        shutil.copy(row['filepath'], os.path.join(label_folder, row['filename']))

copy_images(train_df, 'train')
copy_images(test_df, 'test')

In [6]:
#Visualizing the hierarchy
def print_directory_structure(root_dir, max_files_per_folder=5):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        level = dirpath.replace(root_dir, '').count(os.sep)
        indent = '│   ' * level + '├── '
        print(f'{indent}{os.path.basename(dirpath)}/')
        
        # Print only a few files (up to max_files_per_folder)
        for f in filenames[:max_files_per_folder]:
            print(f'{indent}    {f}')

print_directory_structure('data', max_files_per_folder=3)  # Adjust max_files_per_folder as needed

├── data/
│   ├── train/
│   │   ├── skirt/
│   │   ├──     skirt_464.jpg
│   │   ├──     skirt_568.jpg
│   │   ├──     skirt_994.jpg
│   │   ├── long sleeve top/
│   │   ├──     long_sleeve_top_2328.jpg
│   │   ├──     long_sleeve_top_643.jpg
│   │   ├──     long_sleeve_top_295.jpg
│   │   ├── vest/
│   │   ├──     vest_1251.jpg
│   │   ├──     vest_428.jpg
│   │   ├──     vest_1306.jpg
│   │   ├── long sleeve dress/
│   │   ├──     long_sleeve_dress_997.jpg
│   │   ├──     long_sleeve_dress_2158.jpg
│   │   ├──     long_sleeve_dress_523.jpg
│   │   ├── vest dress/
│   │   ├──     vest_dress_424.jpg
│   │   ├──     vest_dress_1931.jpg
│   │   ├──     vest_dress_1643.jpg
│   │   ├── short sleeve dress/
│   │   ├──     short_sleeve_dress_937.jpg
│   │   ├──     short_sleeve_dress_1806.jpg
│   │   ├──     short_sleeve_dress_831.jpg
│   │   ├── trousers/
│   │   ├──     trousers_1173.jpg
│   │   ├──     trousers_360.jpg
│   │   ├──     trousers_40.jpg
│   │   ├── short sleeve top/
│   │  

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255, # Normalize the pixel values to range [0,1]
    rotation_range = 15, # Random rotations between -15° to +15°
    width_shift_range=0.1,  # Random horizontal shifts (10% of image width)
    height_shift_range=0.1,  # Random vertical shifts (10% of image height)
    shear_range=0.1,  # Random shear transformations
    zoom_range=0.1,  # Random zoom (up to 10%)
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill empty pixels after transformations
)

train_gen = train_datagen.flow_from_directory('data/train', target_size=(256,256), class_mode='categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_gen = test_datagen.flow_from_directory('data/test', target_size=(256,256), class_mode='categorical')

Found 18612 images belonging to 9 classes.
Found 4654 images belonging to 9 classes.


In [ ]:
#Understanding the class weights
print(np.bincount(train_gen.classes))

In [ ]:
print(np.unique(train_gen.classes))  

***Building the model***

In [ ]:
model = Sequential([

    Input(shape=(256,256,3)),

    Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(9, activation='softmax')
])

In [10]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(train_gen.samples)        
print(train_gen.batch_size)     
print(test_gen.samples)
print(test_gen.batch_size)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath='/kaggle/working/my_model_epoch60.keras',      
    monitor='val_loss',             
    save_best_only=True,            
    save_weights_only=False,
    mode='min',                     
    verbose=1                       
)

early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,             
    restore_best_weights=True  
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    min_lr=1e-6
)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(9),  # Assumes you have 9 classes labeled 0 to 8
    y=train_gen.classes     # The actual class labels for each sample in training set
)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
history = model.fit(
    train_gen, 
    epochs=10, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler],
    class_weight=class_weights_dict
)

model.save('/kaggle/working/my_model_epoch10.h5')

In [7]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/mymodel10/keras/default/1/my_model_epoch10.h5')

2025-05-12 11:03:08.546164: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
history = model.fit(
    train_gen, 
    initial_epoch=10,  
    epochs=20, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler],
    class_weight=class_weights_dict
)

model.save('/kaggle/working/my_model_epoch20.h5')

Epoch 11/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


582/582 ━━━━━━━━━━━━━━━━━━━━ 2743s 5s/step - accuracy: 0.5740 - loss: 1.4289 - val_accuracy: 0.7694 - val_loss: 1.0279 - learning_rate: 1.0000e-04
Epoch 12/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2704s 5s/step - accuracy: 0.6159 - loss: 1.3269 - val_accuracy: 0.7572 - val_loss: 1.0293 - learning_rate: 1.0000e-04
Epoch 13/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2671s 5s/step - accuracy: 0.6416 - loss: 1.2336 - val_accuracy: 0.7606 - val_loss: 0.9705 - learning_rate: 1.0000e-04
Epoch 14/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2670s 5s/step - accuracy: 0.6426 - loss: 1.2105 - val_accuracy: 0.8028 - val_loss: 0.7783 - learning_rate: 1.0000e-04
Epoch 15/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2681s 5s/step - accuracy: 0.6594 - loss: 1.1436 - val_accuracy: 0.7684 - val_loss: 0.8139 - learning_rate: 1.0000e-04
Epoch 16/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2718s 5s/step - accuracy: 0.6504 - loss: 1.1623 - val_accuracy: 0.7303 - val_loss: 1.0525 - learning_rate: 1.0000e-04
Epoch 17/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 2680s 5s/step - a

In [9]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/mymodel20/keras/default/1/my_model_epoch20.h5')

2025-05-13 05:05:51.869997: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=5e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_gen, 
    initial_epoch=20,  
    epochs=30, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    class_weight=class_weights_dict
)

Epoch 21/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6959 - loss: 0.9934
Epoch 21: val_loss improved from inf to 0.67996, saving model to /kaggle/working/my_model_epoch30.keras
582/582 ━━━━━━━━━━━━━━━━━━━━ 3140s 5s/step - accuracy: 0.6959 - loss: 0.9934 - val_accuracy: 0.8283 - val_loss: 0.6800 - learning_rate: 5.0000e-05
Epoch 22/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7053 - loss: 0.9595
Epoch 22: val_loss did not improve from 0.67996
582/582 ━━━━━━━━━━━━━━━━━━━━ 3120s 5s/step - accuracy: 0.7053 - loss: 0.9595 - val_accuracy: 0.8388 - val_loss: 0.6909 - learning_rate: 5.0000e-05
Epoch 23/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7170 - loss: 0.9394
Epoch 23: val_loss improved from 0.67996 to 0.67970, saving model to /kaggle/working/my_model_epoch30.keras
582/582 ━━━━━━━━━━━━━━━━━━━━ 3142s 5s/step - accuracy: 0.7170 - loss: 0.9394 - val_accuracy: 0.8260 - val_loss: 0.6797 - learning_rate: 5.0000e-05
Epoch 24/30
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step -

In [8]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/mymodel30/keras/default/1/my_model_epoch30.keras')

2025-05-14 05:11:40.800694: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
history = model.fit(
    train_gen, 
    initial_epoch=30,  
    epochs=40, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    class_weight=class_weights_dict
)

Epoch 31/40


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  6/582 ━━━━━━━━━━━━━━━━━━━━ 1:26:58 9s/step - accuracy: 0.7718 - loss: 0.7670

In [8]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/mymodel40/keras/default/1/my_model_epoch40.keras')

2025-05-16 03:43:09.143760: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=2.5e-5), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
history = model.fit(
    train_gen, 
    initial_epoch=40,  
    epochs=50, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    class_weight=class_weights_dict
)

Epoch 41/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7648 - loss: 0.8150
Epoch 41: val_loss improved from inf to 0.54717, saving model to /kaggle/working/my_model_epoch50.keras
582/582 ━━━━━━━━━━━━━━━━━━━━ 3247s 6s/step - accuracy: 0.7648 - loss: 0.8150 - val_accuracy: 0.8807 - val_loss: 0.5472 - learning_rate: 2.5000e-05
Epoch 42/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7594 - loss: 0.8043
Epoch 42: val_loss did not improve from 0.54717
582/582 ━━━━━━━━━━━━━━━━━━━━ 3270s 6s/step - accuracy: 0.7594 - loss: 0.8043 - val_accuracy: 0.8681 - val_loss: 0.5585 - learning_rate: 2.5000e-05
Epoch 43/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7726 - loss: 0.7800
Epoch 43: val_loss did not improve from 0.54717
582/582 ━━━━━━━━━━━━━━━━━━━━ 3124s 5s/step - accuracy: 0.7726 - loss: 0.7800 - val_accuracy: 0.8655 - val_loss: 0.5484 - learning_rate: 2.5000e-05
Epoch 44/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7833 - loss: 0.7677
Epoch 44: val_loss did not 

In [7]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/mymodel50_/keras/default/1/my_model_epoch50 (1).keras')

2025-05-17 03:53:43.632504: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=6.25e-6), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(
    train_gen, 
    initial_epoch=50,  
    epochs=60, 
    validation_data=test_gen,
    callbacks=[early_stop, lr_scheduler, checkpoint],
    class_weight=class_weights_dict
)

Epoch 51/60


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8127 - loss: 0.6875
Epoch 51: val_loss improved from inf to 0.48592, saving model to /kaggle/working/my_model_epoch60.keras
582/582 ━━━━━━━━━━━━━━━━━━━━ 2646s 5s/step - accuracy: 0.8127 - loss: 0.6875 - val_accuracy: 0.8930 - val_loss: 0.4859 - learning_rate: 6.2500e-06
Epoch 52/60
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8200 - loss: 0.6801
Epoch 52: val_loss improved from 0.48592 to 0.47372, saving model to /kaggle/working/my_model_epoch60.keras
582/582 ━━━━━━━━━━━━━━━━━━━━ 2617s 4s/step - accuracy: 0.8200 - loss: 0.6801 - val_accuracy: 0.8992 - val_loss: 0.4737 - learning_rate: 6.2500e-06
Epoch 53/60
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8195 - loss: 0.6801
Epoch 53: val_loss did not improve from 0.47372
582/582 ━━━━━━━━━━━━━━━━━━━━ 2596s 4s/step - accuracy: 0.8195 - loss: 0.6801 - val_accuracy: 0.8986 - val_loss: 0.4787 - learning_rate: 6.2500e-06
Epoch 54/60
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step -